# Importing Libraries

In [1]:
import sagemaker
import boto3
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.session import s3_input,Session

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [2]:
myregion=boto3.session.Session().region_name
print(myregion)

us-east-1


# Creating New Bucket

In [3]:
# Retrieve the list of existing buckets
s3 = boto3.client('s3')
response = s3.list_buckets()

# Output the bucket names
print('Existing buckets:')
for bucket in response['Buckets']:
    print(f'  {bucket["Name"]}')

Existing buckets:


In [4]:
s3 = boto3.client('s3')
try:
    if myregion == 'us-east-1':
        s3.create_bucket(Bucket='shubham-mg')
        print('Bucket Created Successfully')
except Exception as e:
    print('S3 error:', e)


Bucket Created Successfully


In [5]:
s3 = boto3.client('s3')
response = s3.list_buckets()

# Output the bucket names
print('Existing buckets:')
for bucket in response['Buckets']:
    print(f'  {bucket["Name"]}')

Existing buckets:
  shubham-mg


In [6]:
prefix='xgboost-as-a-built-in-algo'
output_path='s3://{}/{}/output'.format('shubham-mg',prefix)
print(output_path)

s3://shubham-mg/xgboost-as-a-built-in-algo/output


# Downloading and making dataframe of the data 

In [7]:
import pandas as pd
import urllib
try:
    urllib.request.urlretrieve ("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv", "bank_clean.csv")
    print('Success: downloaded bank_clean.csv.')
except Exception as e:
    print('Data load error: ',e)

try:
    model_data = pd.read_csv('./bank_clean.csv',index_col=0)
    print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)

Success: downloaded bank_clean.csv.
Success: Data loaded into dataframe.


# Train-test Split and saving in bucket

In [8]:
import numpy as np 
train_data,test_data=np.split(model_data.sample(frac=1,random_state=42),
                             [int(0.7*len(model_data))])
print(train_data.shape,test_data.shape)

(28831, 61) (12357, 61)


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/numpy/core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [9]:
from sagemaker.inputs import TrainingInput
import pandas as pd
import boto3
import os

# Train Data preparation 
train_data = pd.concat([train_data['y_yes'],
                       train_data.drop(['y_no', 'y_yes'], axis=1)],
                      axis=1)
train_data.to_csv('train.csv', index=False, header=False)

# S3 upload
boto3.Session().resource('s3').Bucket('shubham-mg').Object(
    os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')

# Create TrainingInput
train_input = TrainingInput(
    s3_data='s3://shubham-mg/{}/train'.format(prefix),
    content_type='csv'
)


In [10]:
# Train Data preparation 
test_data = pd.concat([test_data['y_yes'],
                       test_data.drop(['y_no', 'y_yes'], axis=1)],
                      axis=1)
test_data.to_csv('test.csv', index=False, header=False)

# S3 upload
boto3.Session().resource('s3').Bucket('shubham-mg').Object(
    os.path.join(prefix, 'test/test.csv')).upload_file('test.csv')

# Create TrainingInput
test_input = TrainingInput(
    s3_data='s3://shubham-mg/{}/test'.format(prefix),
    content_type='csv'
)


# Building Model XGBoost

In [11]:
from sagemaker import image_uris


# Retrieve the XGBoost container image URI
container = image_uris.retrieve("xgboost", myregion, "1.0-1")


In [12]:
# initialize hyperparameters
hyperparameters = {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
        "objective":"binary:logistic",
        "num_round":50
        }

In [17]:
from sagemaker.estimator import Estimator
estimator = Estimator(
    image_uri=container,  # Specify XGBoost version
    role=sagemaker.get_execution_role(),
    instance_count=1,
    instance_type='ml.m5.2xlarge',
    volume_size=5,
    max_run=300,
    max_wait=600,
    use_spot_instances=True,
    hyperparameters=hyperparameters,
    output_path=output_path,
)


In [18]:
estimator.fit({'train': train_input,'validation': test_input})

INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2024-08-17-14-04-22-440


2024-08-17 14:04:22 Starting - Starting the training job...
2024-08-17 14:04:48 Starting - Preparing the instances for training...
2024-08-17 14:05:09 Downloading - Downloading input data...
2024-08-17 14:05:27 Downloading - Downloading the training image...
2024-08-17 14:06:23 Training - Training image download completed. Training in progress.
2024-08-17 14:06:23 Uploading - Uploading generated training model.[2024-08-17 14:06:17.152 ip-10-0-181-45.ec2.internal:7 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root

# Deploying the model 

In [19]:
xgb_predictor = estimator.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')

INFO:sagemaker:Creating model with name: sagemaker-xgboost-2024-08-17-14-11-25-609
INFO:sagemaker:Creating endpoint-config with name sagemaker-xgboost-2024-08-17-14-11-25-609
INFO:sagemaker:Creating endpoint with name sagemaker-xgboost-2024-08-17-14-11-25-609


------!

# Loading the testing Data 

In [30]:
from sagemaker.predictor import Predictor
from sagemaker.base_serializers import CSVSerializer
import numpy as np

test_data_array = test_data.drop(['y_yes'], axis=1).values  # Load the data into an array
xgb_predictor.content_type = 'csv'
xgb_predictor.serializer = CSVSerializer(content_type='csv') # set the serializer type
# No need to set content_type or serializer explicitly
predictions = xgb_predictor.predict(test_data_array).decode('utf-8')  # Predict!

predictions_array = np.fromstring(predictions[1:], sep=',') # and turn the prediction into an array
print(predictions_array.shape)

(12357,)


# Confusion Matrix

In [31]:
cm = pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))
     


Overall Classification Rate: 89.8%

Predicted      No Purchase    Purchase
Observed
No Purchase    91% (10809)    34% (152)
Purchase        9% (1103)     66% (293) 



# Deleting ALL the data in bucket and deployment endpoint

In [34]:
sagemaker.Session().delete_endpoint(xgb_predictor.endpoint)
bucket_to_delete = boto3.resource('s3').Bucket('shubham-mg')
bucket_to_delete.objects.all().delete()

[{'ResponseMetadata': {'RequestId': 'A64027HQAEA5W3WM',
   'HostId': '5jeNz0IxsehROZVMAu6Sw649dFVY3UGOKTAZl8juYzdwVuFZhZBCcS8ttufCbRJPmvlH9k0evHizw5N6SFJA/atZ8mEBs6sdnk3PaPYdX2c=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': '5jeNz0IxsehROZVMAu6Sw649dFVY3UGOKTAZl8juYzdwVuFZhZBCcS8ttufCbRJPmvlH9k0evHizw5N6SFJA/atZ8mEBs6sdnk3PaPYdX2c=',
    'x-amz-request-id': 'A64027HQAEA5W3WM',
    'date': 'Sat, 17 Aug 2024 14:40:02 GMT',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3',
    'connection': 'close'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'xgboost-as-a-built-in-algo/train/train.csv'},
   {'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2024-08-17-14-04-22-440/profiler-output/system/incremental/2024081714/1723903560.algo-1.json'},
   {'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2024-08-17-14-04-22-440/debug-output/index/000000000/000000000020_worker_0.json'},
   {'Key': 'xgboost-as-a-built